In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data_1 = pd.read_csv("meta_All_Beauty.csv")
data_2 = pd.read_csv("All_Beauty.csv")

In [3]:
data_1.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,NaN
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,NaN
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,NaN
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,NaN
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"['Material: 304 Stainless Steel; Brass tip', '...",['The Precision Plunger Bars are designed to w...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,NaN


In [4]:
data_2.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [5]:
print(data_1.shape)
print(data_2.shape)

(112590, 14)
(701528, 10)


In [6]:
training_data = pd.merge(data_1,data_2, on='parent_asin',how='inner')
training_data.head()

,main_category,title_x,average_rating,rating_number,features,description,price,images_x,videos,store,...,bought_together,rating,title_y,text,images_y,asin,user_id,timestamp,helpful_vote,verified_purchase
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,...,NaN,5.0,Absolutely fabulous - I will never use anythin...,This is amazing. I bought one at a local shop...,[],B01CUPMQZE,AHGJSP3GP5FK37QHS6YEGYTJFG7Q,1490841840000,2,True
1,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,...,NaN,5.0,Of all leather products I have used this one s...,I would recommend this product as the last coa...,[],B01CUPMQZE,AFDEHMXADDA7FFAZORKPABCNI25A,1596490144422,0,True
2,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,...,NaN,4.0,Product works as advertised.,I have a 10 year old Bernhart leather chair an...,[],B01CUPMQZE,AFQZRDEQZSG7UEGWZITJPJBNULVQ,1545653978329,0,True
3,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,...,NaN,5.0,Great stuff,"Works great, our leather sofa has a nice soft ...",[],B01CUPMQZE,AGMXJV5YJ5EJ7QNS7QBZU5M2WNEA,1597709217752,0,True
4,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,...,NaN,4.0,Keeps leather from drying out,I have a leather couch and because we live in ...,[],B01CUPMQZE,AF2RQUPCUOW4PY536TP4JSAVDHOA,1552656079878,0,True


In [7]:
training_data = training_data[['user_id','asin','title_x','average_rating','rating_number','features','description']]

In [8]:
training_data.isnull().sum()

user_id            0
asin               0
title_x           84
average_rating     0
rating_number      0
features           0
description        0
dtype: int64

In [9]:
training_data = training_data.dropna(subset=['title_x'])

In [10]:
training_data.isna().sum()

user_id           0
asin              0
title_x           0
average_rating    0
rating_number     0
features          0
description       0
dtype: int64

In [11]:
training_data.head()

,user_id,asin,title_x,average_rating,rating_number,features,description
0,AHGJSP3GP5FK37QHS6YEGYTJFG7Q,B01CUPMQZE,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[]
1,AFDEHMXADDA7FFAZORKPABCNI25A,B01CUPMQZE,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[]
2,AFQZRDEQZSG7UEGWZITJPJBNULVQ,B01CUPMQZE,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[]
3,AGMXJV5YJ5EJ7QNS7QBZU5M2WNEA,B01CUPMQZE,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[]
4,AF2RQUPCUOW4PY536TP4JSAVDHOA,B01CUPMQZE,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[]


In [12]:
print(training_data.shape)
training_data = training_data.drop_duplicates(subset=['asin'])
training_data = training_data.drop_duplicates(subset=['title_x'])
training_data.shape

(701444, 7)


(110542, 7)

In [13]:
training_data = training_data[~((training_data['features'] == '[]') | (training_data['description'] == '[]'))]

In [14]:
training_data.head()

,user_id,asin,title_x,average_rating,rating_number,features,description
31,AFZHL3O5S7BZ23ADC4H6LZER4R6Q,B07NGFDN6G,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"['Material: 304 Stainless Steel; Brass tip', '...",['The Precision Plunger Bars are designed to w...
32,AFGFAOF5WZHYEMNTID5JZNBWRV7Q,B07G9GWFSM,Lurrose 100Pcs Full Cover Fake Toenails Artifi...,3.7,35,['The false toenails are durable with perfect ...,"['Description', 'The false toenails are durabl..."
41,AFFWXXYYPOOOBB3BX7A2JKYRGVYA,B01ERJEGS6,Gold extatic Musk EDT 90ml,3.7,2,['Extatic Balmain Gold Musk By Balmain Edt Spr...,['Edt spray 3 oz design house: balmain']
58,AFBCAFV2NVBVOF6DXERPJ5HBITOQ,B00FCQHYEW,Small MONEY CLIP Leather Wallet ID Bag Cash Ho...,3.0,6,"['Package:OPP', 'Step 1 : Open Clip and Put Mo...",['Description Small MONEY CLIP Leather Wallet ...
60,AHTRFPOOWXPWICDYRPYADB4WTPNQ,B07TG9LRVR,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52,['GENTLE ON THE SKIN - This bubble face mask c...,"[""This foaming pixie sheet mask is infused wit..."


In [15]:
column_name_mapping = {
    'user_id': 'ID',
    'asin': 'ProdID',
    'title_x': 'Name',
    'average_rating': 'Rating',
    'rating_number': 'ReviewCount',
    'features': 'Features',
    'description': 'Description'
}

training_data.rename(columns=column_name_mapping, inplace=True)
training_data.head()

,ID,ProdID,Name,Rating,ReviewCount,Features,Description
31,AFZHL3O5S7BZ23ADC4H6LZER4R6Q,B07NGFDN6G,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"['Material: 304 Stainless Steel; Brass tip', '...",['The Precision Plunger Bars are designed to w...
32,AFGFAOF5WZHYEMNTID5JZNBWRV7Q,B07G9GWFSM,Lurrose 100Pcs Full Cover Fake Toenails Artifi...,3.7,35,['The false toenails are durable with perfect ...,"['Description', 'The false toenails are durabl..."
41,AFFWXXYYPOOOBB3BX7A2JKYRGVYA,B01ERJEGS6,Gold extatic Musk EDT 90ml,3.7,2,['Extatic Balmain Gold Musk By Balmain Edt Spr...,['Edt spray 3 oz design house: balmain']
58,AFBCAFV2NVBVOF6DXERPJ5HBITOQ,B00FCQHYEW,Small MONEY CLIP Leather Wallet ID Bag Cash Ho...,3.0,6,"['Package:OPP', 'Step 1 : Open Clip and Put Mo...",['Description Small MONEY CLIP Leather Wallet ...
60,AHTRFPOOWXPWICDYRPYADB4WTPNQ,B07TG9LRVR,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52,['GENTLE ON THE SKIN - This bubble face mask c...,"[""This foaming pixie sheet mask is infused wit..."


In [16]:
labelencoder_id = LabelEncoder()
labelencoder_prodid = LabelEncoder()

training_data['ID'] = labelencoder_id.fit_transform(training_data['ID'])

training_data['ProdID'] = labelencoder_prodid.fit_transform(training_data['ProdID'])

In [17]:
training_data.head()

,ID,ProdID,Name,Rating,ReviewCount,Features,Description
31,6007,9063,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"['Material: 304 Stainless Steel; Brass tip', '...",['The Precision Plunger Bars are designed to w...
32,4206,8483,Lurrose 100Pcs Full Cover Fake Toenails Artifi...,3.7,35,['The false toenails are durable with perfect ...,"['Description', 'The false toenails are durabl..."
41,4166,5536,Gold extatic Musk EDT 90ml,3.7,2,['Extatic Balmain Gold Musk By Balmain Edt Spr...,['Edt spray 3 oz design house: balmain']
58,3708,2815,Small MONEY CLIP Leather Wallet ID Bag Cash Ho...,3.0,6,"['Package:OPP', 'Step 1 : Open Clip and Put Mo...",['Description Small MONEY CLIP Leather Wallet ...
60,11496,9457,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52,['GENTLE ON THE SKIN - This bubble face mask c...,"[""This foaming pixie sheet mask is infused wit..."


In [18]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jyoth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def clean_and_extract_tags(text):
    tokens = [word for word in text.lower().split() if word.isalnum() and word not in stop_words]
    return ', '.join(tokens)

training_data['Description'] = training_data['Description'].apply(clean_and_extract_tags)
training_data['Features'] = training_data['Features'].apply(clean_and_extract_tags)

In [20]:
training_data.head()

,ID,ProdID,Name,Rating,ReviewCount,Features,Description
31,6007,9063,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"304, stainless, brass, cartridge, needles, vic...","precision, plunger, bars, designed, work, seam..."
32,4206,8483,Lurrose 100Pcs Full Cover Fake Toenails Artifi...,3.7,35,"false, toenails, durable, perfect, option, wea...","false, toenails, durable, perfect, option, wea..."
41,4166,5536,Gold extatic Musk EDT 90ml,3.7,2,"balmain, gold, musk, balmain, edt, spray, 3","spray, 3, oz, design"
58,3708,2815,Small MONEY CLIP Leather Wallet ID Bag Cash Ho...,3.0,6,"1, open, clip, put, money, closed, clip, open,...","small, money, clip, leather, wallet, id, bag, ..."
60,11496,9457,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52,"skin, bubble, face, mask, contains, nourishing...","foaming, pixie, sheet, mask, infused, nourishi..."


In [21]:
training_data['Tags'] = training_data[['Description','Features']].apply(lambda row: ', '.join(row), axis=1)

In [22]:
training_data.head()

,ID,ProdID,Name,Rating,ReviewCount,Features,Description,Tags
31,6007,9063,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"304, stainless, brass, cartridge, needles, vic...","precision, plunger, bars, designed, work, seam...","precision, plunger, bars, designed, work, seam..."
32,4206,8483,Lurrose 100Pcs Full Cover Fake Toenails Artifi...,3.7,35,"false, toenails, durable, perfect, option, wea...","false, toenails, durable, perfect, option, wea...","false, toenails, durable, perfect, option, wea..."
41,4166,5536,Gold extatic Musk EDT 90ml,3.7,2,"balmain, gold, musk, balmain, edt, spray, 3","spray, 3, oz, design","spray, 3, oz, design, balmain, gold, musk, bal..."
58,3708,2815,Small MONEY CLIP Leather Wallet ID Bag Cash Ho...,3.0,6,"1, open, clip, put, money, closed, clip, open,...","small, money, clip, leather, wallet, id, bag, ...","small, money, clip, leather, wallet, id, bag, ..."
60,11496,9457,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52,"skin, bubble, face, mask, contains, nourishing...","foaming, pixie, sheet, mask, infused, nourishi...","foaming, pixie, sheet, mask, infused, nourishi..."


In [23]:
training_data.dtypes

ID               int32
ProdID           int32
Name            object
Rating         float64
ReviewCount      int64
Features        object
Description     object
Tags            object
dtype: object

In [25]:
training_data.to_csv('ml_projectdata.csv',index=False)

In [26]:
top_rated_items = training_data.sort_values(by='Rating', ascending=False)

In [27]:
top_rated_items.head(10)

,ID,ProdID,Name,Rating,ReviewCount,Features,Description,Tags
250792,6011,6663,Fruit Of The Earth Bogo Cream Shea Butter/Aloe...,5.0,2,"pieces, 4, ounce, cracked, conditions","cracked, softens, conditions","cracked, softens, conditions, pieces, 4, ounce..."
252208,11213,1961,"Natural Life Cosmetic Bag, Owl Speaking Words",5.0,8,"little, things, organization, cotton","little, things, life, cosmetic, speaking, word...","little, things, life, cosmetic, speaking, word..."
400429,11243,6826,Tammy Taylor Gelegance Soak-off Gel 0.5oz/15ml...,5.0,1,"taylor, gelegance, gel, grey","taylor, nail, gelegance, gel, gelegance, gel, ...","taylor, nail, gelegance, gel, gelegance, gel, ..."
465378,7055,4237,Colgate Sensitive Complete Protection Toothpas...,5.0,1,"sensitive, teeth, cavity, healthy, great, tasting","proven, sensitivity, relief, toothpaste, sensi...","proven, sensitivity, relief, toothpaste, sensi..."
251833,1445,1057,Annie Snap On Magnetic Rollers (Black) Size: J...,5.0,1,"snap, magnetic, rollers","snap, magnetic, rollers","snap, magnetic, rollers, snap, magnetic, rollers"
613114,3577,11680,"GelNail Polish24PcsGelNailSet,with1PcsCareGel+...",5.0,4,"season, gel, lasting, good, tenacity, easy, pr...","colors, gel, nail, polish, pacakge, include, 2...","colors, gel, nail, polish, pacakge, include, 2..."
399041,5337,3322,Dakar for MEN Travel Size 0.5 Fl. Oz,5.0,1,"collection, impression, drakkar, classic, frag...","men, travel, size, oz, diamond","men, travel, size, oz, diamond, collection, im..."
201816,4950,4546,Sally Hansen Gem Crush Nail Color COLLECTION O...,5.0,3,"town, sally, hansen, gem, crush, nail, shiny, ...","town, sally, hansen, gem, crush, nail, shiny, ...","town, sally, hansen, gem, crush, nail, shiny, ..."
162203,4232,4749,"ZAHRA BEAUTY Store Spinning Lipstick Towers, A...",5.0,1,"lipstick, lipstick, chic, beautiful, top, easy...","lots, spending, much, time, looking, one, zahr...","lots, spending, much, time, looking, one, zahr..."
397537,6776,2420,WAWO 5 Pcs Nail Art Design Detailing Drawing P...,5.0,2,"art, professional, nail, artist, beginner, exp...","new, high, quality, set, 5, nail, brushes, black","new, high, quality, set, 5, nail, brushes, bla..."


In [28]:
training_data = training_data.reset_index(drop=True)
training_data.head()

,ID,ProdID,Name,Rating,ReviewCount,Features,Description,Tags
0,6007,9063,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"304, stainless, brass, cartridge, needles, vic...","precision, plunger, bars, designed, work, seam...","precision, plunger, bars, designed, work, seam..."
1,4206,8483,Lurrose 100Pcs Full Cover Fake Toenails Artifi...,3.7,35,"false, toenails, durable, perfect, option, wea...","false, toenails, durable, perfect, option, wea...","false, toenails, durable, perfect, option, wea..."
2,4166,5536,Gold extatic Musk EDT 90ml,3.7,2,"balmain, gold, musk, balmain, edt, spray, 3","spray, 3, oz, design","spray, 3, oz, design, balmain, gold, musk, bal..."
3,3708,2815,Small MONEY CLIP Leather Wallet ID Bag Cash Ho...,3.0,6,"1, open, clip, put, money, closed, clip, open,...","small, money, clip, leather, wallet, id, bag, ...","small, money, clip, leather, wallet, id, bag, ..."
4,11496,9457,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52,"skin, bubble, face, mask, contains, nourishing...","foaming, pixie, sheet, mask, infused, nourishi...","foaming, pixie, sheet, mask, infused, nourishi..."


In [ ]:
def cosine_similarity_manual(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (magnitude_vec1 * magnitude_vec2)
    return similarity

def content_based_recommendations_manual(train_data, item_id, top_n=10):
    if item_id not in train_data['ProdID'].values:
        print(f"Item '{item_id}' not found in the training data.")
        return pd.DataFrame()

    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(train_data['Tags']).toarray()
    item_index = train_data[train_data['ProdID'] == item_id].index[0]
    item_vector = tfidf_matrix_content[item_index]
    similarity_scores = []

    for index, vec in enumerate(tfidf_matrix_content):
        similarity = cosine_similarity_manual(item_vector, vec)
        similarity_scores.append((index, similarity))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_similar_items = similarity_scores[1:top_n+1]
    recommended_item_indices = [x[0] for x in top_similar_items]
    recommended_items_details = train_data.iloc[recommended_item_indices][['Name','Rating','ReviewCount']]
    return recommended_items_details


In [38]:
item_id = 4237
content_based_rec_manual = content_based_recommendations_manual(training_data, item_id, top_n=10)

content_based_rec_manual

C:\Users\jyoth\AppData\Local\Temp\ipykernel_15256\378879521.py:10: RuntimeWarning: invalid value encountered in scalar divide
  similarity = dot_product / (magnitude_vec1 * magnitude_vec2)


,Name,Rating,ReviewCount
724,"Charcoal Teeth Whitening Powder,from Organic C...",4.0,115
140,140 Pieces Dental Temporary Crown Front Teeth ...,2.9,408
31,Kidskin- Foaming Body Wash with Tea Tree Oil- ...,4.0,49
5,VIROCHEMISTRY Pheromones For Women (Elixir) - ...,3.7,33
40,Buffing Granules Daily Exfoliating Cleanser - ...,4.8,11
16,Amope PediMask Kit- 20 Minute Foot Mask to Rej...,4.5,1020
21,"Cala Cosmetic wedge travel pack 6 count, 6 Count",1.7,2
12,"Bulk Herbs: Clay, White Kaolin",3.5,3
9,"Enjoy VOLUMIZING ELIXIR, Style (with Sleek Ste...",5.0,1
4,"BioMiracle StarDust Pixie Bubble Mask, Clarify...",4.4,52


In [40]:
content_based_rec_manual.iloc[:2]['Name']

724    Charcoal Teeth Whitening Powder,from Organic C...
140    140 Pieces Dental Temporary Crown Front Teeth ...
Name: Name, dtype: object